In [ ]:
import pandas as pd
from tqdm import tqdm
import sys

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' # (최초 1회)
!pip install chromedriver-autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [ ]:
# selenium 설치 확인
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.31.1


In [40]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [41]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

In [42]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Colab은 새창을 지원하지않기 때문에 창을 띄우지 않는 Headless 모드로 실행해야 합니다.
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

In [43]:
driver = webdriver.Chrome(options=chrome_options)

In [44]:
from bs4 import BeautifulSoup
import re

In [32]:
df = pd.read_excel('네이버 뉴스 제목.xlsx')
df

,Unnamed: 0,날짜,URL,기사 제목
0,0,20240726,/paris2024/article/055/0001175889,"올해 세계선수권 챔피언 김우민, 지난해 우승자 쇼트와 '옆 레인'"
1,1,20240726,/paris2024/article/055/0001175888,테러 우려 속 야외 개회식장으로 가는 길
2,2,20240726,/paris2024/article/016/0002341459,"“너무 비싸다”…사상 최초 센강에서 열리는 개막식, 입장권 가격은 얼마?[파리2024]"
3,3,20240726,/paris2024/article/629/0000308167,[파리 올림픽] 개막 앞두고 프랑스 철도·공항 테러 위협 확산 '긴장'
4,4,20240726,/paris2024/article/016/0002341458,“고속철도 방화에 공항 폭탄 위협까지”…올림픽 개막일에 불안 고조[파리2024]
...,...,...,...,...
27652,27652,20240810,/paris2024/article/629/0000312121,[숏팩트] '2024 파리 올림픽'이 남긴 질문 (영상)
27653,27653,20240810,/paris2024/article/119/0002860783,"‘동메달 결정전만 3번’ 신유빈, 독일 상대로 유종의 미 거둘까"
27654,27654,20240810,/paris2024/article/410/0001017127,"‘아쉽다!’ 서건우, 사상 첫 男 태권도 80kg급 결승행 좌절 [Paris Now]"
27655,27655,20240810,/paris2024/article/055/0001180323,[올림픽] 아쉬운 패배 태권도 서건우…동메달 도전!


In [33]:
df_1 = df.iloc[3000:6916,:]
df_1

,Unnamed: 0,날짜,URL,기사 제목
3000,3000,20240728,/paris2024/article/241/0003368751,"192㎝의 180도 다리 찢기, 오상욱 '예술적 금메달'에 외신도 '감탄' [202..."
3001,3001,20240728,/paris2024/article/055/0001176228,"[올림픽] ""그렇죠 조대성 끌어냅니다!"" 통쾌한 공격…1게임 '결판' 순간 (탁구 ..."
3002,3002,20240728,/paris2024/article/468/0001080673,"‘한국을 북한으로’ 초대형 참사…‘오만한’ 프랑스, 올림픽이 걱정이다 [김동영의 시선]"
3003,3003,20240728,/paris2024/article/410/0001014789,"8년 만의 올림픽 복귀 북한, 2024 파리 올림픽 개막 짤막하게 보도···“특색있..."
3004,3004,20240728,/paris2024/article/421/0007692536,'韓선수단 최연소' 반효진 올림픽新…공기소총 10m 1위로 '결선행'[올림픽]
...,...,...,...,...
6911,6911,20240730,/paris2024/article/056/0011772216,"김민섭, 한국 접영 최초 준결승 진출…황선우 100m 준결승행"
6912,6912,20240730,/paris2024/article/241/0003369307,"황선우, 자유형 100m 예선 턱걸이 통과…16명 중 16위로 준결승행 [2024 파리]"
6913,6913,20240730,/paris2024/article/056/0011772215,[영상] ‘금지수’ 되고픈 ‘김지수’…세계 1위 꺾고 8강 갔다
6914,6914,20240730,/paris2024/article/449/0000281348,2조 썼는데…센강 수질 악화에 경기 연기


In [45]:
from tqdm.auto import tqdm
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd
from bs4 import BeautifulSoup

# URL 리스트 생성
url_list = df_1['URL']
full_url_list = url_list.map(lambda x: 'https://m.sports.naver.com' + x)

data = []

# URL마다 내용을 크롤링
for url in tqdm(full_url_list):
    driver.get(url)
    time.sleep(3)

    try:
        # 특정 XPath를 사용하여 요소 찾기
        element = driver.find_element(By.XPATH, '//*[@id="comp_news_article"]')

        # 요소의 HTML 가져오기
        html_content = element.get_attribute('outerHTML')

        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(html_content, 'html.parser')

        # 해당 요소의 텍스트만 추출
        content_str = soup.get_text()

        # 데이터 리스트에 추가
        data.append((url, content_str))

    except NoSuchElementException:
        # 요소를 찾을 수 없을 경우 예외 처리
        print(f"Element not found for URL: {url}")
        data.append((url, None))  # 찾을 수 없는 경우 None을 추가

# DataFrame으로 변환
content_df = pd.DataFrame(data, columns=['URL', '본문'])


  0%|          | 0/3916 [00:00<?, ?it/s]

Element not found for URL: https://m.sports.naver.com/paris2024/article/214/0001364029
Element not found for URL: https://m.sports.naver.com/paris2024/article/421/0007691919
Element not found for URL: https://m.sports.naver.com/paris2024/article/025/0003376166


In [48]:
content_df

,URL,본문
0,https://m.sports.naver.com/paris2024/article/2...,오상욱의 다리찢기. 게티이미지오상욱이 금메달을 향해 다리를 쭉 찢었다. 완벽한 다리...
1,https://m.sports.naver.com/paris2024/article/0...,\n안내본 동영상은 저작권자인 언론사의 요청에 따라국내에서만 재생할 수 있습니다.해...
2,https://m.sports.naver.com/paris2024/article/4...,대한민국 올림픽 선수단의 기수 우상혁(오른쪽)과 김서영이 26일(현지시간) 프랑스 ...
3,https://m.sports.naver.com/paris2024/article/4...,◆ 2024 파리올림픽 ◆북한이 7월 27일 프랑스 파리 올림픽 개막 소식을 다음날...
4,https://m.sports.naver.com/paris2024/article/4...,"17세 고교생, 634.5점으로 본선 44명 중 1위29일 오후 4시30분 메달 도..."
...,...,...
3911,https://m.sports.naver.com/paris2024/article/0...,2024 파리올림픽에서 수영의 김민섭이 한국 수영 사상 최초로 올림픽 접영 준결승 ...
3912,https://m.sports.naver.com/paris2024/article/2...,100m 자유형 예선 출전한 황선우 (파리=연합뉴스) 김인철 기자 = 30일(현지...
3913,https://m.sports.naver.com/paris2024/article/0...,\n안내본 동영상은 저작권자인 언론사의 요청에 따라국내에서만 재생할 수 있습니다.해...
3914,https://m.sports.naver.com/paris2024/article/4...,[앵커]이번 파리 올림픽 이례적으로 센강에서 개막식 했죠. 자연과 함께하겠다며 ...


In [49]:
content_df.to_excel('네이버 뉴스 본문(3000~6915).xlsx')
